# We import the libraries

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

# We read the data  

In [15]:
def read_data(filename):

    with open(filename, 'r') as file:
        distance_matrix = False
        city_list = False
        for line in file:
            line = line.strip()
            # Create the distance matrix and the list of cities
            if line.startswith('DIMENSION'):
                dimension = int(line.split()[-1])
                cities = np.zeros((dimension, 2))
                distances = np.zeros((dimension, dimension))
                i=0
                j=0
            
            # What do we do with the line
            if line.startswith('EDGE_WEIGHT_SECTION'):
                distance_matrix = True
                continue
            elif line.startswith('DISPLAY_DATA_SECTION'):
                distance_matrix = False
                city_list = True
                continue
            elif line.startswith('EOF'):
                break
            
            # Save data
            if distance_matrix:
                distances[i,] = [int(x) for x in line.split()]
                i += 1
            if city_list:
                cities[j,] = line.split()[-2:]
                j += 1
        
        return distances, cities

In [16]:
distances,cities= read_data('bays29.tsp')

ValueError: invalid literal for int() with base 10: '1150.0'

In [7]:
distances[1][2]

np.int64(148)